In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
enst_tpms = []

for sample_id, fastqs in SETTING["fastq_id_fastq_file_paths"].items():

    enst_tpm = pd.read_csv(
        "{}/{}/abundance.tsv".format(PATH["kallisto/"], sample_id),
        sep="\t",
        index_col=0,
    )["tpm"]

    enst_tpm.name = sample_id

    enst_tpms.append(enst_tpm)

enst_x_sample = pd.concat(enst_tpms, axis=1)

enst_x_sample.index.name = "ENST"

enst_x_sample.to_csv(PATH["enst_x_sample.tsv"], sep="\t")

enst_x_sample

In [ ]:
enst_gene_name = pd.read_csv(SETTING["enst_gene_name_file_path"], sep="\t")

gene_x_sample = pd.DataFrame(
    enst_x_sample,
    index=enst_x_sample.index.map(
        dict(
            zip(
                enst_gene_name["Transcript stable ID version"],
                enst_gene_name["Gene name"].str.upper(),
            )
        )
    ),
)

gene_x_sample.index.name = "Gene"

gene_x_sample__processed = ccal.process_feature_x_sample(
    gene_x_sample.loc[~gene_x_sample.index.isna()].groupby(level=0).mean(),
    features_to_drop=SETTING["features_to_drop"],
    samples_to_drop=SETTING["samples_to_drop"],
    nanize=SETTING["nanize"],
    drop_axis=SETTING["drop_axis"],
    max_na=SETTING["max_na"],
    min_n_not_na_unique_value=SETTING["min_n_not_na_unique_value"],
    shift_as_necessary_to_achieve_min_before_logging=SETTING[
        "shift_as_necessary_to_achieve_min_before_logging"
    ],
    log_base=SETTING["log_base"],
    normalization_axis=SETTING["normalization_axis"],
    normalization_method=SETTING["normalization_method"],
    clip_min=SETTING["clip_min"],
    clip_max=SETTING["clip_max"],
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_distributions_max_size=SETTING["plot_distributions_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)

gene_x_sample__processed.to_csv(PATH["gene_x_sample.tsv"], sep="\t")

gene_x_sample__processed